# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
from numpy import mean
from numpy import absolute
from numpy import sqrt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score,mean_squared_error,r2_score

# Importing Datasets

In [3]:
train=pd.read_csv("train_dataset.csv")

In [4]:
test=pd.read_csv("test_dataset.csv")

# Extracting Data

In [5]:
train=train.set_index('Unnamed: 0')
X_train=train.iloc[:,0:9]
y_train=train.Survived

In [6]:
test=test.set_index('Unnamed: 0')
X_test=test.iloc[:,0:9]
y_test=test.Survived

# Taking a look at the data

In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(534, 9)
(534,)
(179, 9)
(179,)


In [8]:
X_train.head()

,Pclass,Sex,Age,Fare,Embarked,FamilyCount,Title,Has Cabin,IsAlone
Unnamed: 0,,,,,,,,,
711,1,0,29.699118,26.5500,0,0,1,1,0
466,2,0,29.699118,0.0000,0,0,1,0,0
213,2,0,30.000000,13.0000,0,0,1,0,0
496,1,1,54.000000,78.2667,2,1,2,1,1
583,1,0,36.000000,40.1250,2,0,1,1,0


In [9]:
y_train.head()

Unnamed: 0
711    0
466    0
213    0
496    1
583    0
Name: Survived, dtype: int64

# KNN Model

In [76]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
acc_knn = accuracy_score(y_test,y_pred)
acc_knn

0.770949720670391

# Hyperparameter Tuning to improve Accuracy

In [83]:
from sklearn.model_selection import GridSearchCV
n_neighbors = list(range(1,17,2))
weights = ['uniform','distance']
metric = ['minkowski','euclidean','manhattan']
leaf_size = list(range(1,15))
grid_params = { 'n_neighbors' : n_neighbors,
               'weights' : weights,
               'metric' : metric,
               'leaf_size' : leaf_size}

In [84]:
KNNModel_grid = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)
g_res = KNNModel_grid.fit(X_train, y_train)

Fitting 3 folds for each of 672 candidates, totalling 2016 fits


In [85]:
g_res.best_params_

{'leaf_size': 1,
 'metric': 'manhattan',
 'n_neighbors': 7,
 'weights': 'distance'}

In [86]:
g_res.best_score_

0.7471910112359551

# Cross validating the model

In [87]:
#define cross-validation method to use
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [88]:
scores = cross_val_score(KNNModel_grid, X_train, y_train, scoring='neg_mean_absolute_error',cv=cv, n_jobs=-1)

#view mean absolute error
mean(absolute(scores))

0.2657931516422082

In [89]:
#define cross-validation method to use
cv = KFold(n_splits=5, random_state=1, shuffle=True) 

#build knn model
knn=KNeighborsClassifier()

#use LOOCV to evaluate model
scores = cross_val_score(KNNModel_grid, X_train, y_train, scoring='neg_mean_squared_error',cv=cv, n_jobs=-1)

#view RMSE
sqrt(mean(absolute(scores)))

0.5103064036823535

Both MAE and RMSE are not that low meaning our model's performane is not that good

# Pickle the model

In [90]:
# save the model to disc
import pickle
filename = "KNN model.pkl"
pickle.dump(KNNModel_grid,open(filename,'wb'))